In [1]:
import os
from tensorflow import keras
import losses, train, GhostFaceNets
import tensorflow as tf
import keras_cv_attention_models

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4060 Ti, compute capability 8.9


GhostFaceNetV1

In [4]:
# (MS1MV2) dataset
data_path = 'datasets/all_data_112x112_folders'
eval_paths = ['datasets/all_data/lfw.bin', 'datasets/all_data/cfp_fp.bin', 'datasets/all_data/agedb_30.bin']

In [5]:
# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation --> activation
>>>> Convert ReLU: activation_1 --> activation_1
>>>> Convert ReLU: activation_2 --> activation_2
>>>> Convert ReLU: activation_3 --> activation_3
>>>> Convert ReLU: activation_4 --> activation_4
>>>> Convert ReLU: activation_5 --> activation_5
>>>> Convert ReLU: activation_6 --> activation_6
>>>> Convert ReLU: activation_7 --> activation_7
>>>> Convert ReLU: activation_8 --> activation_8
>>>> Convert ReLU: activation_9 --> activation_9
>>>> Convert ReLU: activation_11 --> activation_11
>>>> Convert ReLU: activation_12 --> activation_12
>>>> Convert ReLU: activation_13 --> activation_13
>>>> Convert ReLU: activation_15 --> activation_15
>>>> Convert ReLU: activation_16 --> activation_16
>>>> Convert ReLU: activation_17 --> activation_17
>>>> Convert ReLU: activation_18 --> activation_18
>>>> Convert ReLU: activation_19 --> activation_19
>>>> Convert ReLU: activation_20 --> ac

In [6]:
# # Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2_new_data_(8file_githud).h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)


>>>> L2 regularizer value from basic_model: 0.00025


In [7]:
print(dir(tt))  # Kiểm tra xem 'train' có nằm trong danh sách không


['__add_emb_output_to_model__', '__basic_train__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_dataset__', '__init_emb_losses__', '__init_model__', '__init_optimizer__', '__init_subclass__', '__init_type_by_loss__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__search_embedding_layer__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'arcface', 'arcface_partial', 'basic_model', 'batch_size', 'batch_size_per_replica', 'center', 'classes', 'custom_callbacks', 'data_path', 'default_optimizer', 'distill', 'distill_emb_map_layer', 'gently_stop', 'image_per_class', 'inited_from_model', 'is_distill_ds', 'is_triplet_dataset', 'lr_scheduler', 'metrics', 'mixup_alpha', 'model', 'model_checkpoint', 'my_evals', 'my_history', 'output_weight_decay', 'partial_fc_split', 'pretrained', 'random_cutout_mask_are

In [8]:

optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 8},
]
# tt.steps_per_epoch = 10
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> reloaded from dataset backup: all_data_112x112_folders_shuffle.npz
>>>> Image length: 1219995, Image class length: 1219995, classes: 109999
>>>> Use specified optimizer: <keras.optimizer_v2.gradient_descent.SGD object at 0x00000199931207F0>
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 1, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': -9531, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}

Learning rate for iter 1 is 0.10000000149011612, global_iterNum is 0
9531/9531 [==============================] - ETA: 0s - loss: 179.7921 - accuracy: 0.2717


Evaluating lfw: 100%|██████████| 94/94 [00:34<00:00,  2.74it/s]


>>>> lfw evaluation max accuracy: 0.824167, thresh: 0.564626, previous max accuracy: 0.000000
>>>> Improved = 0.824167




Evaluating cfp_fp: 100%|██████████| 110/110 [00:39<00:00,  2.77it/s]


>>>> cfp_fp evaluation max accuracy: 0.624714, thresh: 0.478383, previous max accuracy: 0.000000
>>>> Improved = 0.624714



...


Evaluating agedb_30: 100%|██████████| 94/94 [00:34<00:00,  2.73it/s]


>>>> agedb_30 evaluation max accuracy: 0.500500, thresh: 0.909635, previous max accuracy: 0.560333



c:\Users\ADMIN\anaconda3\envs\tf\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Epoch 00009: saving model to checkpoints\ghostnetv1_w1.3_s2_new_data_(8file_githud)_epoch09.h5
9531/9531 [==============================] - 5327s 559ms/step - loss: 78.7983 - accuracy: 0.8212
>>>> Train arcface DONE!!! epochs = [1, 2, 3, 4, 5, 6, 7, 8], model.stop_training = False
>>>> My history:
{
  'lr': [0.09987824410200119, 0.09951350092887878, 0.09890756756067276, 0.09806337207555771, 0.09698506444692612, 0.09567786008119583, 0.09414813667535782, 0.0924033597111702, 0.09045201539993286],
  'loss': [116.6863899230957, 192.51508331298828, 55.7662353515625, 42.73808288574219, 43.11186981201172, 39.31891632080078, 36.74291229248047, 35.22093200683594, 33.14404296875],
  'accuracy': [0.2717415392398834, 0.690266489982605, 0.7569797039031982, 0.7775375843048096, 0.7917215824127197, 0.8009636402130127, 0.8085744976997375, 0.814972996711731, 0.8212363123893738],
  'lfw': [0.8241666666666667, 0.8013333333333333, 0.82, 0.8088333333333333, 0.8076666666666666, 0.7861666666666667, 0.81133333

9

In [9]:
print("done train")

done train
